# Crash Analysis

In [1]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu
import subprocess
import requests
from tempfile import TemporaryDirectory

In [2]:
API_TOKEN = open("api_token.txt").read().strip()
API_BASE  = "https://rode0day.mit.edu/api/1.0/"
COMP_DIR  = "."

## AFL-CMIN Validation

In [12]:
program_arguments = {
    'objdump'   : ['objdump', '-D','@@']
,    'nm'       : ['nm-new', '-C','@@']
,    'ffmpeg'   : ['ffmpeg','-i','@@']
,    'gif2png'  : ['gif2png','@@']
,    'mupdf'    : ['mutool','show','@@']
,    'size'     : ['size','@@']
,    'tcpdump' : ['tcpdump', '-nr','@@']
,    'readelf'  : ['readelf','-a','@@']
}
rodeo_arguments = {
    'grep': ['grep', '-f'],
    'memdjpeg': ['memdjpeg'],
    'newgrep': ['newgrep','-f'],
    'bzip': ['bzip', '-c']
}

base_folder = '/home/deif/gits/fuzztestbench/'

In [4]:
program_arguments[0]

KeyError: 0

In [63]:
program = 'nm'
program_call = program_arguments[program]
program_call[0] = base_folder + program + 'pre/' + program_call[0]
fuzzer = 'curiousmutation'

In [13]:
crashes = {
    'program': [],
    'fuzzer': [],
    'crashes': []
}
fuzzers = ['afl', 'aflfast', 'curiousmutation']
for program in program_arguments:
    
    program_call = program_arguments[program]
    program_call[0] = base_folder + program + 'pre/' + program_call[0]

    for fuzzer in fuzzers:
        for i in range(1, 6):
            with TemporaryDirectory(dir='./') as temp_dir: #create temp outputdir for every run
                indir = os.path.join(base_folder, program + 'pre/' + program + 'pre' + fuzzer + str(i) + '/afl_out/crashes/')
                if os.path.isdir(indir) and len(os.listdir(indir))>0:
                    #print(['afl-cmin', '-i', indir] + ['-o', temp_dir] + program_call)
                    out = subprocess.run(['afl-cmin', '-C', '-i', indir] + ['-o', temp_dir + '/'] + program_call)
                    #print(os.listdir(temp_dir))
                    crashes['program'].append(program)
                    crashes['fuzzer'].append(fuzzer)
                    crashes['crashes'].append(len(os.listdir(temp_dir)))
                else:
                    crashes['program'].append(program)
                    crashes['fuzzer'].append(fuzzer)
                    crashes['crashes'].append(0)

df_crashes = pd.DataFrame(crashes)
df_crashes.to_csv('./crashes_real_programs.csv')

### real programs

In [29]:
df_crashes = pd.DataFrame(crashes)
for program in df_crashes['program'].unique():
    afl_mean = df_crashes.loc[(df_crashes.program==program) & (df_crashes.fuzzer=='afl')]['crashes'].mean()
    aflfast_mean = df_crashes.loc[(df_crashes.program==program) & (df_crashes.fuzzer=='aflfast')]['crashes'].mean()
    curiousafl_mean = df_crashes.loc[(df_crashes.program==program) & (df_crashes.fuzzer=='curiousmutation')]['crashes'].mean()
    if not (afl_mean == 0.0 and aflfast_mean == 0.0 and curiousafl_mean == 0.0):
        print(program + '\t' + 'afl: ' + str(afl_mean)+ '\t' + 'aflfast: ' + str(aflfast_mean)+ '\t' + 'curiousafl: ' + str(curiousafl_mean) )

objdump	afl: 0.0	aflfast: 0.0	curiousafl: 1.0
nm	afl: 11.0	aflfast: 11.8	curiousafl: 66.6
ffmpeg	afl: 102.8	aflfast: 112.2	curiousafl: 51.0
gif2png	afl: 1.0	aflfast: 5.0	curiousafl: 40.0
size	afl: 0.2	aflfast: 3.2	curiousafl: 0.0


In [13]:
#! echo " " >> ./test
challenge_id = 14
input_file = os.path.join(COMP_DIR, "test")
r = requests.post(API_BASE+"submit", data={"challenge_id": challenge_id, "auth_token": API_TOKEN}, files={"input": input_file})

## Real programs validation

In [2]:
program_arguments = {
    'objdump'   : ['objdump', '-D']
,    'nm'       : ['nm', '-C']
,    'ffmpeg'   : 'ffmpeg -i'
,    'gif2png'  : ['gif2png', '']
,    'mupdf'    : 'mutool show'
,    'size'     : 'size'
,    'harfbuzz' : 'hb-fuzzer' 
,    'readelf'  : 'readelf -a'
}


In [41]:
program = 'gif2png'
program_argument = program_arguments[program]

for i in range(1,6):
    crash_path = '/home/deif/gits/fuzztestbench/' + program + 'pre/' + program + 'precuriousmutation' + str(i) + '/'
    for filename in sorted(os.listdir(crash_path + 'afl_out/crashes/')):
        out = subprocess.run(program_argument + [os.path.join(crash_path + 'afl_out/crashes/', filename)])
        if out.returncode != 0:
            print(out)

CompletedProcess(args=['gif2png', '', '/home/deif/gits/fuzztestbench/gif2pngpre/gif2pngprecuriousmutation1/afl_out/crashes/README.txt'], returncode=2)
CompletedProcess(args=['gif2png', '', '/home/deif/gits/fuzztestbench/gif2pngpre/gif2pngprecuriousmutation1/afl_out/crashes/id:000000,sig:11,src:000068,op:havoc,rep:128'], returncode=2)
CompletedProcess(args=['gif2png', '', '/home/deif/gits/fuzztestbench/gif2pngpre/gif2pngprecuriousmutation1/afl_out/crashes/id:000001,sig:11,src:000115,op:havoc,rep:8'], returncode=2)
CompletedProcess(args=['gif2png', '', '/home/deif/gits/fuzztestbench/gif2pngpre/gif2pngprecuriousmutation1/afl_out/crashes/id:000002,sig:11,src:000195,op:havoc,rep:32'], returncode=2)
CompletedProcess(args=['gif2png', '', '/home/deif/gits/fuzztestbench/gif2pngpre/gif2pngprecuriousmutation1/afl_out/crashes/id:000003,sig:11,src:000133,op:havoc,rep:8'], returncode=2)
CompletedProcess(args=['gif2png', '', '/home/deif/gits/fuzztestbench/gif2pngpre/gif2pngprecuriousmutation1/afl_out

FileNotFoundError: [Errno 2] No such file or directory: '/home/deif/gits/fuzztestbench/gif2pngpre/gif2pngprecuriousmutation5/afl_out/crashes/'